The goal of this ipynb notebook is to attempt to achieve better performance on predicting hourly relative humidity through 

1) Harmonic Regression
2) A Neural Network

In [95]:
import os
import numpy as np
import pandas as pd
import torch 
import torch.nn as nn
import torch.optim as optim
import re
from tqdm import tqdm

import subprocess
from datetime import datetime, timedelta
from concurrent.futures import ProcessPoolExecutor
import argparse
import sys

from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score
import math
import random

Import the data frame

In [132]:
data_path = "/Volumes/Seagate Portable Drive/Relative_Humidity_Project/5.modeling_data/10_5_2025_Data/RH_Prediction_Data.xlsx"
data = pd.read_excel(data_path)

We ultimately want to use maxRH and minRH to predict HourlyRelativeHumidity. In the initial linear models, we used:

model1 <- feglm(RelativeHumidity ~ maxRH*doy + maxRH*year + maxRH*era.id + 
                  minRH*doy + minRH*year + minRH*era.id, data = hour1, family = "gaussian")

model2 <- feglm(RelativeHumidity ~ maxRH*doy + maxRH*year + maxRH*era.id + 
                 minRH*doy + minRH*year + minRH*era.id, data = hour2, family = "gaussian")

Thus, for modeling, we will use maxRH, minRH, year, era.id, and doy

In [155]:
#Subset the data
model_data = data[["maxRH", "minRH", "year", "era.id", "doy", "Hour", "HourlyRelativeHumidity"]]

#Get rid of NAs
model_data = model_data.dropna(subset = ["HourlyRelativeHumidity"])
model_data = model_data[model_data["HourlyRelativeHumidity"] != "*"]

Process the data

In [156]:
#Perform cyclical encoding for feature engineering
model_data["Hour_sin"] = np.sin(2 * np.pi * model_data["Hour"] / 24.0)
model_data["Hour_cos"] = np.cos(2 * np.pi * model_data["Hour"] / 24.0)
model_data["doy_sin"] = np.sin(2 * np.pi * model_data["doy"] / 365.0)
model_data["doy_cos"] = np.cos(2 * np.pi * model_data["doy"] / 365.0)

#Now turn era.id into a categorical variable
era_cat = model_data["era.id"].astype("category")
model_data["era_code"] = era_cat.cat.codes.astype(np.int64)
n_era = len(model_data["era_code"].unique())

In [ ]:
#Now we select the features and split them
num_cols = ["maxRH", "minRH", "year", "Hour_sin", "Hour_cos", "doy_sin", "doy_cos"]
cat_cols = "era_code"
target = "HourlyRelativeHumidity"
model_data[target] = pd.to_numeric(model_data[target], errors = "coerce")

#Split before scaling
train_df, test_df = train_test_split(model_data, test_size = 0.2, random_state = 42, stratify = model_data["era_code"])
train_df, val_df = train_test_split(train_df, test_size = 0.2, random_state = 42, stratify = train_df["era_code"])

train_df = train_df.drop(columns = ["era_code"])
val_df = val_df.drop(columns = ["era_code"])
test_df = test_df.drop(columns = ["era_code"])

#Now we scale the numeric features
scaler = StandardScaler().fit(train_df[num_cols].values)
def transform_num(df):
    return scaler.transform(df[num_cols].values).astype(np.float32)

#Now we transform the data
Xnum_train = transform_num(train_df)
Xnum_val = transform_num(val_df)
Xnum_test = transform_num(test_df)

y_train = train_df[target].values.astype(np.float32)
y_val = val_df[target].values.astype(np.float32)
y_test = test_df[target].values.astype(np.float32)

Now we put the data sets into Torch format

In [ ]:
class RHDataSet(Dataset):
    def __init__(self, Xnum, y):
        self.Xnum = torch.from_numpy(Xnum)
        # self.Xcat = torch.from_numpy(Xcat)
        self.y = torch.from_numpy(y)

    def __len__(self): return len(self.y)
    def __getitem__(self, idx):
        # return self.Xnum[idx], self.Xcat[idx], self.y[idx]
        return self.Xnum[idx], self.y[idx]


#Now we convert them
ds_train = RHDataSet(Xnum_train, y_train)
ds_val = RHDataSet(Xnum_val, y_val)
ds_test = RHDataSet(Xnum_test, y_test)

dl_train = DataLoader(ds_train, batch_size = 256, shuffle = True, num_workers = 0)
dl_val = DataLoader(ds_val, batch_size = 512, shuffle = False, num_workers = 0)
dl_test = DataLoader(ds_test, batch_size = 512, shuffle = False, num_workers = 0)

Now we model

In [159]:
class RHNet(nn.Module):
    def __init__(self, n_num, hidden = (128, 64), p = 0.1):
        super().__init__()
        layers, last = [], n_num
        for h in hidden:
            layers += [nn.Linear(last, h), nn.ReLU(), nn.Dropout(p)]
            last = h
        layers += [nn.Linear(last, 1)]
        self.mlp = nn.Sequential(*layers)
    
    def forward(self, x_num):
        return self.mlp(x_num).squeeze(1)

model = RHNet(n_num = len(num_cols)).to("cpu")

Now we train the model

In [160]:
criterion = nn.MSELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr = 1e-3, weight_decay = 1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode = "min", patience = 5, factor = 0.5, verbose = False)

patience, patience_ctr = 15, 0
epochs = 200

def run_epoch(model, loader, train_mode = True):
    model.train(train_mode)
    total_loss = 0.0
    n = 0

    loop = tqdm(loader, leave = False, mininterval = 0.5)
    if epoch is not None:
        loop.set_description(f"{'Train' if train_mode else 'Val'} Epoch {epoch}")

    with torch.set_grad_enabled(train_mode):
        # for xb_num, xb_cat, yb in loader:
        for xb_num, yb in loader:
            xb_num = xb_num.to("cpu")
            # xb_cat = xb_cat.to("cpu")
            yb = yb.to("cpu")

            if train_mode:
                optimizer.zero_grad()

            # preds = model(xb_num, xb_cat)
            preds = model(xb_num)
            loss = criterion(preds, yb)

            if train_mode:
                loss.backward()
                optimizer.step()
            
            total_loss += loss.item() * yb.size(0)
            n += yb.size(0)
    
    return total_loss / max(n, 1)

/Users/kchan/.pyenv/versions/tsfm_env/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


Train the model

In [161]:
best_val = float("inf")
best_state = None

for epoch in range(1, epochs + 1):
    train_loss = run_epoch(model, dl_train, train_mode = True)
    val_loss = run_epoch(model, dl_val, train_mode = False)
    scheduler.step(val_loss)
    print(f"Val Loss is: {val_loss:.3f}")

    if val_loss < best_val - 1e-6:
        best_val = val_loss
        best_state = {k: v.cpu().clone() for k, v in model.state_dict().items()}
        patience_ctr = 0
    else:
        patience_ctr += 1
    
    if patience_ctr >= patience:
        break

if best_state is not None:
    model.load_state_dict(best_state)
else:
    print("Warning: No improement detected - using final model weights")

model.to("cpu")

Val Loss is: 100.281


Val Loss is: 92.504


Val Loss is: 91.300


Val Loss is: 89.647


Val Loss is: 89.116


...

Val Loss is: 85.049


Val Loss is: 85.047


Val Loss is: 85.037


Val Loss is: 85.030


RHNet(
  (mlp): Sequential(
    (0): Linear(in_features=7, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.1, inplace=False)
    (3): Linear(in_features=128, out_features=64, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.1, inplace=False)
    (6): Linear(in_features=64, out_features=1, bias=True)
  )
)

Now we evaluate performance

In [ ]:
def predict(model, loader):
    model.eval()
    outs, trues = [], []
    with torch.no_grad():
        for xb_num, yb in loader:
            xb_num = xb_num.to("cpu")
            preds = model(xb_num).cpu().numpy()
            outs.append(preds)
            trues.append(yb.numpy())
        
    yhat = np.concatenate(outs)
    ytrue = np.concatenate(trues)

    #Clip to valid RH range
    yhat = np.clip(yhat, 0.0, 100.0)
    return ytrue, yhat

y_true, y_pred = predict(model, dl_test)
mae = mean_absolute_error(y_true, y_pred)
rmse = math.sqrt(((y_true - y_pred) ** 2).mean())
r2 = r2_score(y_true, y_pred)

print(f"Test MAE: {mae:0.3f}")
print(f"Test RMSE: {rmse:0.3f}")
print(f"Test R2: {r2:0.3f}")

Test MAE: 6.917
Test RMSE: 9.224
Test R2: 0.797
